<img src="./img/Logo.png" align="right" width="50%"></img>
# Mission Atlantic GeoNode Workshop
# How to use OGC webservices offered by the Mission Atlantic GeoNode in your data analysis

## 1. Search through metadata using the OGC Catalogue Service (CSW)

The OWSLib library is a python library that allows for accessing data and metadata through OGC services. The library abstracts all of the traditional HTTP API calls into programmatic function calls. We will be using this library throughout the tutorial to interact with the MissionAtlantic GeoNode OGC services.

The library is available from GitHub @  https://github.com/geopython/OWSLib

In [1]:
from owslib.csw import CatalogueServiceWeb

#### Create a CatalogueServiceWeb object

In [2]:
csw = CatalogueServiceWeb('https://geonode.missionatlantic.eu/catalogue/csw')

#### Inspect its properties using print()

In [3]:
print(csw.identification.type)
print(csw.identification.title)
print(csw.identification.version)
print([op.name for op in csw.operations])

CSW
GeoNode Catalogue
2.0.2
['GetCapabilities', 'DescribeRecord', 'GetDomain', 'GetRecords', 'GetRecordById', 'GetRepositoryItem']


#### Inspect the supported GetDomain parameters

In [4]:
csw.get_operation_by_name("GetDomain").parameters

{'ParameterName': {'values': ['DescribeRecord.outputFormat',
   'DescribeRecord.schemaLanguage',
   'DescribeRecord.typeName',
   'GetCapabilities.sections',
   'GetRecordById.ElementSetName',
   'GetRecordById.outputFormat',
   'GetRecordById.outputSchema',
   'GetRecords.CONSTRAINTLANGUAGE',
   'GetRecords.ElementSetName',
   'GetRecords.outputFormat',
   'GetRecords.outputSchema',
   'GetRecords.resultType',
   'GetRecords.typeNames']}}

#### Get supported constraint languages

In [5]:
csw.getdomain('GetRecords.CONSTRAINTLANGUAGE')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.CONSTRAINTLANGUAGE',
 'values': ['CQL_TEXT', 'FILTER']}

#### Get the supported elementsets

In [6]:
csw.getdomain('GetRecords.ElementSetName')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.ElementSetName',
 'values': ['brief', 'full', 'summary']}

#### Get supported output formats

In [7]:
csw.getdomain('GetRecords.outputFormat')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.outputFormat',
 'values': ['application/json', 'application/xml']}

#### Get supported output schemas

In [8]:
csw.getdomain('GetRecords.outputSchema')
csw.results

{'type': 'csw:Record',
 'parameter': 'GetRecords.outputSchema',
 'values': ['http://gcmd.gsfc.nasa.gov/Aboutus/xml/dif/',
  'http://www.interlis.ch/INTERLIS2.3',
  'http://www.isotc211.org/2005/gmd',
  'http://www.opengis.net/cat/csw/2.0.2',
  'http://www.opengis.net/cat/csw/csdgm',
  'http://www.w3.org/2005/Atom',
  'urn:oasis:names:tc:ebxml-regrep:xsd:rim:3.0']}

#### Search data by using OGC Filter Encoding

In [9]:
from owslib.fes import PropertyIsEqualTo, PropertyIsLike

>##### Example: search for AnyText fields that equal "Lophelia"
'AnyText' searches for the term within any of the text fields in the datasets

In [10]:
anytext_query = PropertyIsEqualTo('apiso:AnyText','Lophelia')
csw.getrecords2(constraints=[anytext_query], maxrecords=20, esn='full',outputschema='http://www.isotc211.org/2005/gmd')
print(csw.results)
for rec in csw.records:
    print(csw.records[rec].identification[0].title)
    print(csw.records[rec].identification[0].abstract)
    print("----")

{'matches': 3, 'returned': 3, 'nextrecord': 0}
Modeled distribution map of Lophelia Reef and Pheronema aggregations in the North East Atlantic in 2022 - Pheronema carpenteri
Species distribution models (Maxent) predicting the distribution of two Vulnerable Marine Ecosystems (VME): the reef-forming Scleractinian coral Desmophyllum pertusum and the aggregations forming Hexactinellid sponge Pheronema carpenteri. Both of these species are VME indicator taxa and form habitat that enhance deep-sea diversity (Ross and Howell, 2013). Maps of the likely distribution of the habitat formed by these two species will enable efficient Marine Spatial planning to facilitate their conservation. This work was performed at the University of Plymouth in 2021. A GIS layer is provided for each species.
----
Modeled distribution map of Lophelia Reef and Pheronema aggregations in the North East Atlantic in 2022 - Desmophyllum pertusum
Species distribution models (Maxent) predicting the distribution of two Vul

> ##### Example: search for title fields that contain "Classification"
'Title' searches for the term solely within dataset titles

In [11]:
title_query = PropertyIsLike('apiso:Title','%classification%', wildCard="%")
csw.getrecords2(constraints=[title_query], maxrecords=20, esn='full',outputschema='http://www.isotc211.org/2005/gmd')
print(csw.results)
for rec in csw.records:
    print(csw.records[rec].identification[0].title)
    print(csw.records[rec].identification[0].abstract)
    print("----")

{'matches': 5, 'returned': 5, 'nextrecord': 0}
Benthic habitats using EUNIS classification for the Brazilian EEZ
The dataset contains the shapefile of marine benthic broad habitats classified according to EUNIS classification for the Brazilian Exclusive Economic Zone region. It was developed using open access databases and GIS operations to develop two intermediate layers: (i) biological zones and (ii) substrate type. Both layers were combined to form the first continuous map of marine habitats for the entire region.
----
Hierarchical benthic habitat classification in the Atlantic Ocean - Level 1
Classification of the Atlantic Ocean seabed into broad-scale benthic habitats employing a hierarchical top-down clustering approach aimed at informing Marine Spatial Planning. This work was performed at the University of Plymouth in 2021 with data provided by a wide group of partners representing the nations surrounding the Atlantic Ocean. It classifies continuous environmental data into discr

#### Search with a CQL query
CQL allows to combine search term and filters to create more elaborate queries, narrowing down down the results

> Example: search for Lophelia and 2021 in AnyText fields

In [12]:
csw.getrecords2(cql='csw:AnyText like "%Lophelia%" AND dc:title like "%2021%"',maxrecords=20,esn='full',outputschema='http://www.isotc211.org/2005/gmd')  
print(csw.results)              # For more information on query syntax, please refer to: https://developer.atlassian.com/server/confluence/advanced-searching-using-cql/ . 
for rec in csw.records:
    print(csw.records[rec].identification[0].title)
    print(csw.records[rec].identification[0].abstract)
    print("----")

{'matches': 1, 'returned': 1, 'nextrecord': 0}
Modeled distribution map of discrete Lophelia pertusa colonies in the North East Atlantic in 2021
Species distribution models (GAM, Maxent and Random Forest ensemble) predicting the distribution of discrete Lophelia pertusa - Desmophylum pertusum colonies assemblage in the Celtic Sea. This community is considered ecologically coherent according to the cluster analysis conducted by Parry et al. (2015) on image samples. Modelling its distribution complements existing work on their definition and offers a representation of the extent of the areas of the North East Atlantic where they can occur based on the best available knowledge. This work was performed at the University of Plymouth in 2021.
----


#### Get record metadata

In [13]:
from pprint import pprint
records = [csw.records[rec] for rec in csw.records]
record = records[0]
pprint(vars(record.identification[0]))

{'abstract': 'Species distribution models (GAM, Maxent and Random Forest '
             'ensemble) predicting the distribution of discrete Lophelia '
             'pertusa - Desmophylum pertusum colonies assemblage in the Celtic '
             'Sea. This community is considered ecologically coherent '
             'according to the cluster analysis conducted by Parry et al. '
             '(2015) on image samples. Modelling its distribution complements '
             'existing work on their definition and offers a representation of '
             'the extent of the areas of the North East Atlantic where they '
             'can occur based on the best available knowledge. This work was '
             'performed at the University of Plymouth in 2021.',
 'abstract_url': None,
 'accessconstraints': [],
 'aggregationinfo': None,
 'alternatetitle': None,
 'bbox': <owslib.iso.EX_GeographicBoundingBox object at 0x0000020110C07170>,
 'classification': [],
 'contact': [<owslib.iso.CI_Responsibl

#### Get record data

In [14]:
for resource in record.distribution.online:
    print('Description: ', resource.description)
    print('Protocol: ', resource.protocol)
    print('URL: ', resource.url)
    print("---")

Description:  Online link to the 'Modeled distribution map of discrete Lophelia pertusa colonies in the North East Atlantic in 2021' description on GeoNode
Protocol:  WWW:LINK-1.0-http--link
URL:  https://geonode.missionatlantic.eu/layers/DisLop_Ens_Cont_Preds_Thresh_200m_GHO_Masked_reprojectedGeoNode:geonode:DisLop_Ens_Cont_Preds_Thresh_200m_GHO_Masked_reprojectedGeoNode
---
Description:  Modeled distribution map of discrete Lophelia pertusa colonies in the North East Atlantic in 2021 (Thumbnail Format)
Protocol:  WWW:DOWNLOAD-1.0-http--download
URL:  https://geonode.missionatlantic.eu/uploaded/thumbs/layer-a14d8add-73ea-4056-956f-8a59c2baab61-thumb-4b55c202-b4fb-4888-ac02-ed04f96eae0c.png
---
Description:  Modeled distribution map of discrete Lophelia pertusa colonies in the North East Atlantic in 2021 (Legend Format)
Protocol:  WWW:DOWNLOAD-1.0-http--download
URL:  https://geonode.missionatlantic.eu/geoserver/ows?service=WMS&request=GetLegendGraphic&format=image/png&WIDTH=20&HEIGHT=

### [>> Next: Visualize data using OGC Web Mapping Service (WMS)](./2_visualize_data_with_WMS.ipynb) 

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>